<a href="https://colab.research.google.com/github/Samortchy/AI-projects/blob/main/Video-To-Text%20Summrization/Summrization_attempt_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
import random
from rouge_score import rouge_scorer

nltk.download('stopwords')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/data science/cnn_dailymail/train.csv')

df = df.dropna().sample(n=5000, random_state=42)
df = df.rename(columns={'article': 'text', 'highlights': 'summary'})


texts = df['text'].tolist()
summaries = df['summary'].tolist()


In [ ]:
df.head()

,id,text,summary
272581,ed0fed726929c1eeabe6c390e47128dbb7d7a055,By . Mia De Graaf . Britons flocked to beaches...,<sos>People enjoyed temperatures of 17C at Bri...
772,023cd84001b33aed4ff0f3f5ecb0fdd2151cf543,A couple who weighed a combined 32st were sham...,<sos>Couple started piling on pounds after the...
171868,6a70a0d8d3ed365fe1df6d35f1587a8b9b298618,Video footage shows the heart stopping moment ...,<sos>A 17-year-old boy suffering lacerations t...
63167,b37204c13ea38b511265e41ac69fb12acfb63f85,"Istanbul, Turkey (CNN) -- About 250 people rac...",<sos>Syrians citizens hightail it to Turkey .\...
68522,c24e5805afd5145bc48410e876db91d44a06be5e,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,<sos>The Xue Long had provided the helicopter ...


In [ ]:
#Tokenizers
text_tokenizer = Tokenizer(oov_token='<OOV>')
summary_tokenizer = Tokenizer(oov_token='<OOV>')

# Include <sos> and <eos> explicitly
summary_tokenizer.fit_on_texts(['<sos>', '<eos>'])
summary_tokenizer.fit_on_texts(summaries)
text_tokenizer.fit_on_texts(texts)

# Sequences
text_seq = text_tokenizer.texts_to_sequences(texts)
summary_seq = summary_tokenizer.texts_to_sequences(summaries)

# Max lengths
max_text_len = 300
max_summary_len = 50

# Pad sequences
encoder_input = pad_sequences(text_seq, maxlen=max_text_len, padding='post')
decoder_input = pad_sequences(summary_seq, maxlen=max_summary_len, padding='post')

In [ ]:
print("<sos>" in summary_tokenizer.word_index)  # Should return: True
#print(summary_tokenizer.word_index['<sos>'])    # Should return an integer index


False


In [ ]:
# Model params
embedding_dim = 100
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(max_text_len,))
enc_emb = Embedding(len(text_tokenizer.word_index)+1, embedding_dim)(encoder_inputs)
encoder_lstm, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(len(summary_tokenizer.word_index)+1, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(len(summary_tokenizer.word_index)+1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Compile model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
# Prepare decoder output (shifted by one timestep)
decoder_target = np.zeros_like(decoder_input)
decoder_target[:, :-1] = decoder_input[:, 1:]

# Train
model.fit(
    [encoder_input, decoder_input],
    np.expand_dims(decoder_target, -1),
    batch_size=64,
    epochs=10,
    validation_split=0.1
)

Epoch 1/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 656s 9s/step - loss: 8.6684 - val_loss: 6.9857
Epoch 2/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 704s 9s/step - loss: 6.9331 - val_loss: 6.7881
Epoch 3/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 681s 9s/step - loss: 6.6888 - val_loss: 6.6081
Epoch 4/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 672s 9s/step - loss: 6.5422 - val_loss: 6.5798
Epoch 5/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 672s 9s/step - loss: 6.4751 - val_loss: 6.5772
Epoch 6/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 687s 9s/step - loss: 6.5026 - val_loss: 6.5776
Epoch 7/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 674s 9s/step - loss: 6.4716 - val_loss: 6.5513
Epoch 8/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 684s 9s/step - loss: 6.4963 - val_loss: 6.5393
Epoch 9/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 681s 9s/step - loss: 6.4235 - val_loss: 6.5181
Epoch 10/10
71/71 ━━━━━━━━━━━━━━━━━━━━ 696s 9s/step - loss: 6.3860 - val_loss: 6.4903


In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_states2 = [state_h2, state_c2]

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq, encoder_model, decoder_model, tokenizer, max_len):
    states_value = encoder_model.predict(input_seq)

    # Start with the most frequent token or simply 0 (usually padding token)
    target_seq = np.zeros((1, 1))

    decoded_tokens = []
    stop_condition = False

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, '')

        # Stop if too long or unknown/pad token
        if len(decoded_tokens) >= max_len or sampled_word == '':
            stop_condition = True
        else:
            decoded_tokens.append(sampled_word)

        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return ' '.join(decoded_tokens)


In [ ]:
print(df['summary'].iloc[32])  # Make sure it's exactly "<sos> ... <eos>"
print(repr(df['summary'].iloc[32]))  # Reveals hidden characters


<sos>Heather Choate found out she had breast cancer while 10 weeks pregnant with her sixth child .
The 29-year-old from Bayfield, Colorado was advised to terminate the pregnancy and seek treatment, but said she would rather die .
She found a doctor who gave her some chemotherapy and surgery, and gave birth to a healthy baby girl on January 6 .
Earlier this week, she also learned that she is now cancer free .<eos>
'<sos>Heather Choate found out she had breast cancer while 10 weeks pregnant with her sixth child .\nThe 29-year-old from Bayfield,\xa0Colorado\xa0was advised to terminate the pregnancy and\xa0seek treatment, but said she would rather die .\nShe found a doctor who gave her some chemotherapy and surgery, and gave birth to a\xa0healthy\xa0baby girl on\xa0January\xa06 .\nEarlier this week, she also learned that she is now cancer free .<eos>'


In [ ]:
input_seq = encoder_input[0].reshape(1, -1)
generated = decode_sequence(
    input_seq, encoder_model, decoder_model, summary_tokenizer, max_summary_len
)

print("\nOriginal Text:")
print(texts[5][:500], '...')
print("\nReference Summary:")
print(summaries[5])
print("\nGenerated Summary:")
print(generated)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step

Original Text:
(CNN) -- Place a tennis ball into a yellow sock and tuck it into the back of your pants. Now start running. Congratulations! You are a snitch. Don't get caught, or the game ends. This is quidditch, as played by muggles. In JK Rowling's novels, the snitch is a sort-of magical, winged squash ball, pursued by "seeker" Harry Potter in a variety of high-stakes, airborne quidditch contests. Nobody on the UCLA campus has a magical, winged squash ball. A man with a tennis ball dangling from his shorts w ...

Reference Summary:
<sos>'Muggle quidditch' replicates Harry Potter's magical sport in the real world .
Invented in 2005, quidditch now supports 170 US college teams and five full-time staff .
Documentary follows UCLA's quidditch team to the Quidditch World Cup .
Quidditch players torn between developing their sport and preserving its roots .<eos>

Generated Summary:



In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

scores = scorer.score(summaries[0], generated)
print(scores)